# Test d'Exécution - Audit Biais et Carbone

Ce notebook teste l'intégration des audits de **biais** et de **carbone** avec le système d'exécutions du backend.

## Prérequis
- Backend Django en cours d'exécution sur `http://localhost:8000`
- Package `auraagent` installé (`pip install -e .`)
- Variables d'environnement configurées

In [ ]:
# Imports
import os
import time
from aura import BiasAnalyzer, AuraCarbon

✅ Imports réussis !
📍 Répertoire actuel: /Users/vtombou/Desktop/CEVIA/aura/v1


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/codecarbon/core/gpu.py:4: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml


## 1️⃣ Configuration de l'environnement

Configurons les variables d'environnement pour se connecter au backend.

In [ ]:
# Configuration backend
BACKEND_URL = "http://localhost:8000"
API_KEY = "wZFIXkg_xtApj9SE3uMpLkH7PsUcTqlw2QXrtf_41eM"  # Remplacer par votre clé API si nécessaire

# ID de l'exécution à tester (à remplacer par un ID réel)
EXECUTION_ID = "run-20260223-055250"  # Exemple

print(f"🔗 Backend URL: {BACKEND_URL}")
print(f"🎯 Execution ID: {EXECUTION_ID}")

🔗 Backend URL: http://localhost:8000
🎯 Execution ID: b3f0936a-75b0-415c-ad8c-74e23365c329


## 2️⃣ Test Audit de Biais

Testons l'analyse de biais avec un modèle simple.

In [3]:
# Définir un modèle simple pour le test
def simple_model(question: str) -> str:
    """
    Modèle de test qui répond toujours 'A'.
    
    Dans un cas réel, vous remplacerez ceci par votre modèle d'IA.
    Le modèle doit retourner "A", "B", "C" ou "D".
    """
    return "A"

# Test du modèle
print("🧪 Test du modèle :")
test_q = "What is the best answer?"
print(f"   Question : {test_q}")
print(f"   Réponse  : {simple_model(test_q)}")

🧪 Test du modèle :
   Question : What is the best answer?
   Réponse  : A


In [ ]:
# Lancer l'analyse de biais
print("🔧 Initialisation de BiasAnalyzer...")
print()

# BiasAnalyzer charge automatiquement la config depuis ~/.aura.config
analyzer = BiasAnalyzer()  # ✅ Pas de paramètres !

print("🚀 Lancement de l'analyse de biais...")
print("   (Cela prendra environ 10-30 secondes pour 2 tests/catégorie)")
print()

bias_results = analyzer.analyze(
    model_callable=simple_model,
    model_name="Test Notebook Model",
    number_of_tests=2,  # 2 × 6 catégories = 12 tests (rapide)
    track_carbon=False,  # On teste le carbone séparément
    verbose=True,
    execution_id=EXECUTION_ID  # ✅ execution_id va dans analyze() !
)

print(f"\n✅ Analyse terminée ! {bias_results['total_tests_run']} tests exécutés")

In [ ]:
# Afficher les résultats de biais
print("\n" + "=" * 80)
print("📊 RÉSULTATS ANALYSE DE BIAIS")
print("=" * 80)
print()
print(f"🎯 Score global de biais : {bias_results['overall_bias_score']:.2f}")
print(f"📈 Tests exécutés        : {bias_results['total_tests_run']}")
print(f"❌ Tests échoués         : {bias_results['total_tests_failed']}")
print()
print("📊 Scores par catégorie :")
for category, score in bias_results['scores_by_category'].items():
    emoji = "✅" if score < 30 else "⚠️" if score < 60 else "❌"
    print(f"   {emoji} {category:20s} : {score:.2f}%")

print()
print("🔗 Consultez les détails sur le dashboard :")
print(f"   {BACKEND_URL}/audits/executions/{EXECUTION_ID}/")

## 3️⃣ Test Audit Carbone

Testons le tracking carbone avec quelques calculs.

In [ ]:
# Configuration du tracker carbone
print("🔥 Test Carbon Tracking...")
print("-" * 60)

# AuraCarbon charge aussi la config depuis ~/.aura.config
carbon_tracker = AuraCarbon(
    project_name="test-notebook-carbon",
    execution_id=EXECUTION_ID  # ✅ Utilise automatiquement ~/.aura.config
)

print("✅ Tracker carbone configuré")

In [ ]:
# Lancer le tracking avec un context manager
with carbon_tracker as tracker:
    print("⚙️  Calculs en cours...")
    
    # Simulation de calculs
    result = sum(i ** 2 for i in range(5_000_000))
    time.sleep(2)  # Simulation d'un traitement
    
    print(f"   Résultat: {result}")
    print("   ✅ Calculs terminés")

print("\n✅ Tracking carbone terminé !")
print(f"🔗 Vérifiez le dashboard : {BACKEND_URL}/audits/executions/{EXECUTION_ID}/")

## 4️⃣ Vérification de la progression

Vérifions que la progression de l'exécution a été mise à jour.

In [ ]:
import requests

# Récupérer les infos de l'exécution via l'API
try:
    response = requests.get(f"{BACKEND_URL}/api/executions/{EXECUTION_ID}/")
    if response.status_code == 200:
        execution_data = response.json()
        print("📊 État de l'exécution :")
        print(f"   Progression : {execution_data.get('progress_percentage', 0)}%")
        print(f"   Statut      : {execution_data.get('status', 'unknown')}")
        print(f"   Axes actifs : {execution_data.get('active_axes', [])}")
    else:
        print(f"⚠️ Impossible de récupérer l'exécution (status: {response.status_code})")
        print("   Vérifiez manuellement sur le dashboard")
except Exception as e:
    print(f"⚠️ Erreur lors de la vérification: {e}")
    print("   Vérifiez manuellement sur le dashboard")

## 5️⃣ Résumé

Tests complétés ! Vérifiez maintenant le dashboard pour voir :

1. **Audit de biais** : Score global et détails par catégorie
2. **Audit carbone** : Émissions CO2 et consommation énergétique  
3. **Progression** : Pourcentage de complétion de l'exécution

🔗 **Dashboard** : http://localhost:8000/audits/executions/{EXECUTION_ID}/

In [ ]:
# Afficher le lien direct
print("\n" + "=" * 80)
print("🎉 TESTS TERMINÉS !")
print("=" * 80)
print()
print(f"🔗 Accéder au dashboard :")
print(f"   {BACKEND_URL}/audits/executions/{EXECUTION_ID}/")
print()
print("✅ Audit de biais : Complété")
print("✅ Audit carbone  : Complété")
print()
print("📊 Vérifiez la progression sur le dashboard !")